In [1]:
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
imageLocation = bs.find('a', {'id': 'logo'}).find('img')['src']
urlretrieve (imageLocation, 'logo.jpg')

('logo.jpg', <http.client.HTTPMessage at 0x107b427b8>)

In [2]:
import os
from urllib.request import urlretrieve
from urllib.request import urlopen
from bs4 import BeautifulSoup

downloadDirectory = 'downloaded'
baseUrl = 'http://pythonscraping.com'

def getAbsoluteURL(baseUrl, source):
    if source.startswith('http://www.'):
        url = 'http://{}'.format(source[11:])
    elif source.startswith('http://'):
        url = source
    elif source.startswith('www.'):
        url = source[4:]
        url = 'http://{}'.format(source)
    else:
        url = '{}/{}'.format(baseUrl, source)
    if baseUrl not in url:
        return None
    return url

def getDownloadPath(baseUrl, absoluteUrl, downloadDirectory):
    path = absoluteUrl.replace('www.', '')
    path = path.replace(baseUrl, '')
    path = downloadDirectory+path
    directory = os.path.dirname(path)

    if not os.path.exists(directory):
        os.makedirs(directory)

    return path

html = urlopen('http://www.pythonscraping.com')
bs = BeautifulSoup(html, 'html.parser')
downloadList = bs.findAll(src=True)

for download in downloadList:
    fileUrl = getAbsoluteURL(baseUrl, download['src'])
    if fileUrl is not None:
        print(fileUrl)

urlretrieve(fileUrl, getDownloadPath(baseUrl, fileUrl, downloadDirectory))

http://pythonscraping.com/misc/jquery.js?v=1.4.4
http://pythonscraping.com/misc/jquery.once.js?v=1.2
http://pythonscraping.com/misc/drupal.js?os2esm
http://pythonscraping.com/sites/all/themes/skeletontheme/js/jquery.mobilemenu.js?os2esm
http://pythonscraping.com/sites/all/modules/google_analytics/googleanalytics.js?os2esm
http://pythonscraping.com/sites/default/files/lrg_0.jpg
http://pythonscraping.com/img/lrg%20(1).jpg


('downloaded/img/lrg%20(1).jpg', <http.client.HTTPMessage at 0x107b827f0>)

In [1]:
import csv

csvFile = open('test.csv', 'w+')
try:
    writer = csv.writer(csvFile)
    writer.writerow(('number', 'number plus 2', 'number times 2'))
    for i in range(10):
        writer.writerow( (i, i+2, i*2))
finally:
    csvFile.close()

In [4]:
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs = BeautifulSoup(html, 'html.parser')
#The main comparison table is currently the first table on the page
table = bs.findAll('table',{'class':'wikitable'})[0]
rows = table.findAll('tr')

csvFile = open('editors.csv', 'wt+')
writer = csv.writer(csvFile)
try:
    for row in rows:
        csvRow = []
        for cell in row.findAll(['td', 'th']):
            csvRow.append(cell.get_text())
            writer.writerow(csvRow)
finally:
    csvFile.close()

In [7]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import pymysql
import re

conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
                       user='root', passwd=None, db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute("USE scraping")

random.seed(datetime.datetime.now())

def store(title, content):
    cur.execute('INSERT INTO pages (title, content) VALUES ("%s", "%s")', (title, content))
    cur.connection.commit()

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org'+articleUrl)
    bs = BeautifulSoup(html, 'html.parser')
    title = bs.find('h1').get_text()
    content = bs.find('div', {'id':'mw-content-text'}).find('p').get_text()
    store(title, content)
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
try:
    while len(links) > 0:
         newArticle = links[random.randint(0, len(links)-1)].attrs['href']
         print(newArticle)
         links = getLinks(newArticle)
finally:
    cur.close()
    conn.close()

OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 2] No such file or directory)")

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pymysql
from random import shuffle

conn = pymysql.connect(host='127.0.0.1', unix_socket='/tmp/mysql.sock',
                       user='root', passwd='root', db='mysql', charset='utf8')
cur = conn.cursor()
cur.execute('USE wikipedia')

def insertPageIfNotExists(url):
    cur.execute('SELECT * FROM pages WHERE url = %s', (url))
    if cur.rowcount == 0:
        cur.execute('INSERT INTO pages (url) VALUES (%s)', (url))
        conn.commit()
        return cur.lastrowid
    else:
        return cur.fetchone()[0]

def loadPages():
    cur.execute('SELECT * FROM pages')
    pages = [row[1] for row in cur.fetchall()]
    return pages

def insertLink(fromPageId, toPageId):
    cur.execute('SELECT * FROM links WHERE fromPageId = %s AND toPageId = %s', 
                  (int(fromPageId), int(toPageId)))
    if cur.rowcount == 0:
        cur.execute('INSERT INTO links (fromPageId, toPageId) VALUES (%s, %s)', 
                    (int(fromPageId), int(toPageId)))
        conn.commit()
def pageHasLinks(pageId):
    cur.execute('SELECT * FROM links WHERE fromPageId = %s', (int(pageId)))
    rowcount = cur.rowcount
    if rowcount == 0:
        return False
    return True

def getLinks(pageUrl, recursionLevel, pages):
    if recursionLevel > 4:
        return

    pageId = insertPageIfNotExists(pageUrl)
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    links = bs.findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))
    links = [link.attrs['href'] for link in links]

    for link in links:
        linkId = insertPageIfNotExists(link)
        insertLink(pageId, linkId)
        if not pageHasLinks(linkId):
            print("PAGE HAS NO LINKS: {}".format(link))
            pages.append(link)
            getLinks(link, recursionLevel+1, pages)
        
        
getLinks('/wiki/Kevin_Bacon', 0, loadPages()) 
cur.close()
conn.close()

1
462
1705
0
PAGE HAS NO LINKS: /wiki/Pennsylvania
0
PAGE HAS NO LINKS: /wiki/Pennsylvania_(disambiguation)
1
0
PAGE HAS NO LINKS: /wiki/Province_of_Pennsylvania
0
PAGE HAS NO LINKS: /wiki/Kingdom_of_England
0
PAGE HAS NO LINKS: /wiki/England
0
PAGE HAS NO LINKS: /wiki/Commonwealth_of_England
0
PAGE HAS NO LINKS: /wiki/Flag_of_England
0
PAGE HAS NO LINKS: /wiki/Royal_Arms_of_England
0
PAGE HAS NO LINKS: /wiki/Royal_Banner_of_England
0
PAGE HAS NO LINKS: /wiki/Winchester
0
PAGE HAS NO LINKS: /wiki/Westminster
0
PAGE HAS NO LINKS: /wiki/City_of_London
0
PAGE HAS NO LINKS: /wiki/Old_Norse
0
PAGE HAS NO LINKS: /wiki/Welsh_language
0
PAGE HAS NO LINKS: /wiki/Cornish_language
0
PAGE HAS NO LINKS: /wiki/Cumbric_language
0
PAGE HAS NO LINKS: /wiki/Anglo-Norman_language
0
PAGE HAS NO LINKS: /wiki/Medieval_Latin
0
PAGE HAS NO LINKS: /wiki/Christianity
0
PAGE HAS NO LINKS: /wiki/Demonym
0
PAGE HAS NO LINKS: /wiki/English_people
0
PAGE HAS NO LINKS: /wiki/Absolute_monarchy
0
PAGE HAS NO LINKS: /wi

0
PAGE HAS NO LINKS: /wiki/History_of_Portsmouth
0
PAGE HAS NO LINKS: /wiki/History_of_Reading,_Berkshire
0
PAGE HAS NO LINKS: /wiki/History_of_Rochester,_Kent
0
PAGE HAS NO LINKS: /wiki/History_of_Sheffield
0
PAGE HAS NO LINKS: /wiki/History_of_Shrewsbury
0
PAGE HAS NO LINKS: /wiki/History_of_Southampton
0
PAGE HAS NO LINKS: /wiki/History_of_St_Albans
0
PAGE HAS NO LINKS: /wiki/History_of_Torquay
0
PAGE HAS NO LINKS: /wiki/History_of_Wetherby
0
PAGE HAS NO LINKS: /wiki/History_of_Worthing
0
PAGE HAS NO LINKS: /wiki/History_of_York
0
PAGE HAS NO LINKS: /wiki/Sovereign_state
0
PAGE HAS NO LINKS: /wiki/Island
0
PAGE HAS NO LINKS: /wiki/Great_Britain
0
PAGE HAS NO LINKS: /wiki/History_of_Anglo-Saxon_England
0
PAGE HAS NO LINKS: /wiki/Kingdom_of_Scotland
0
PAGE HAS NO LINKS: /wiki/Kingdom_of_Great_Britain
0
PAGE HAS NO LINKS: /wiki/%C3%86thelstan
0
PAGE HAS NO LINKS: /wiki/North_Sea_Empire
0
PAGE HAS NO LINKS: /wiki/Cnut_the_Great
0
PAGE HAS NO LINKS: /wiki/Denmark
0
PAGE HAS NO LINKS: /wi

0
PAGE HAS NO LINKS: /wiki/English_claims_to_the_French_throne
0
PAGE HAS NO LINKS: /wiki/Hundred_Years%27_War
0
PAGE HAS NO LINKS: /wiki/House_of_Plantagenet
0
PAGE HAS NO LINKS: /wiki/Capetian_House_of_Valois
0
PAGE HAS NO LINKS: /wiki/Battle_of_Formigny
0
PAGE HAS NO LINKS: /wiki/Battle_of_Castillon
0
PAGE HAS NO LINKS: /wiki/Calais
0
PAGE HAS NO LINKS: /wiki/English_identity
0
PAGE HAS NO LINKS: /wiki/Anglo-Saxon
0
PAGE HAS NO LINKS: /wiki/Joan_of_Arc
0
PAGE HAS NO LINKS: /wiki/Anglo-Normans
0
PAGE HAS NO LINKS: /wiki/Middle_English
0
PAGE HAS NO LINKS: /wiki/Wars_of_the_Roses
0
PAGE HAS NO LINKS: /wiki/House_of_Lancaster
0
PAGE HAS NO LINKS: /wiki/House_of_York
0
PAGE HAS NO LINKS: /wiki/Henry_VII_of_England
0
PAGE HAS NO LINKS: /wiki/Elizabeth_of_York
0
PAGE HAS NO LINKS: /wiki/Tudor_dynasty
0
PAGE HAS NO LINKS: /wiki/Tudor_period
0
PAGE HAS NO LINKS: /wiki/Elizabethan_era
0
PAGE HAS NO LINKS: /wiki/Stuart_period
0
PAGE HAS NO LINKS: /wiki/English_Renaissance
0
PAGE HAS NO LINKS:

0
PAGE HAS NO LINKS: /wiki/Anglo-Norman_England
0
PAGE HAS NO LINKS: /wiki/Angevin_Empire
0
PAGE HAS NO LINKS: /wiki/England_in_the_Late_Middle_Ages
0
PAGE HAS NO LINKS: /wiki/Wars_of_the_Roses
0
PAGE HAS NO LINKS: /wiki/Tudor_period
0
PAGE HAS NO LINKS: /wiki/Stuart_period_(England)
0
PAGE HAS NO LINKS: /wiki/English_Renaissance
0
PAGE HAS NO LINKS: /wiki/English_Reformation
0
PAGE HAS NO LINKS: /wiki/Elizabethan_era
0
PAGE HAS NO LINKS: /wiki/Union_of_the_Crowns
0
PAGE HAS NO LINKS: /wiki/English_overseas_possessions
0
PAGE HAS NO LINKS: /wiki/Gunpowder_Plot
0
PAGE HAS NO LINKS: /wiki/Jacobean_era
0
PAGE HAS NO LINKS: /wiki/English_Civil_War
0
PAGE HAS NO LINKS: /wiki/Interregnum_(England)
0
PAGE HAS NO LINKS: /wiki/Commonwealth_of_England
0
PAGE HAS NO LINKS: /wiki/The_Protectorate
0
PAGE HAS NO LINKS: /wiki/Restoration_(England)
0
PAGE HAS NO LINKS: /wiki/Popish_Plot
0
PAGE HAS NO LINKS: /wiki/Glorious_Revolution
0
PAGE HAS NO LINKS: /wiki/Maritime_history_of_England
0
PAGE HAS NO 

0
PAGE HAS NO LINKS: /wiki/Kingdom_of_Ireland
0
PAGE HAS NO LINKS: /wiki/Kingdom_of_Scotland
0
PAGE HAS NO LINKS: /wiki/United_Kingdom_of_Great_Britain_and_Ireland
0
PAGE HAS NO LINKS: /wiki/Germanic_languages
0
PAGE HAS NO LINKS: /wiki/English_language
0
PAGE HAS NO LINKS: /wiki/Scots_language
0
PAGE HAS NO LINKS: /wiki/Celtic_languages
0
PAGE HAS NO LINKS: /wiki/Cornish_language
0
PAGE HAS NO LINKS: /wiki/Scottish_Gaelic
0
PAGE HAS NO LINKS: /wiki/Irish_language
0
PAGE HAS NO LINKS: /wiki/Manx_language
0
PAGE HAS NO LINKS: /wiki/Welsh_language
0
PAGE HAS NO LINKS: /wiki/Romance_languages
0
PAGE HAS NO LINKS: /wiki/Auregnais
0
PAGE HAS NO LINKS: /wiki/French_language
0
PAGE HAS NO LINKS: /wiki/Guern%C3%A9siais
0
PAGE HAS NO LINKS: /wiki/J%C3%A8rriais
0
PAGE HAS NO LINKS: /wiki/Sercquiais
0
PAGE HAS NO LINKS: /wiki/British_Sign_Language
0
PAGE HAS NO LINKS: /wiki/Irish_Sign_Language
0
PAGE HAS NO LINKS: /wiki/Northern_Ireland_Sign_Language
0
PAGE HAS NO LINKS: /wiki/Shelta
0
PAGE HAS N

...

0
PAGE HAS NO LINKS: /wiki/Jamestown,_Virginia
0
PAGE HAS NO LINKS: /wiki/Bermuda
0
PAGE HAS NO LINKS: /wiki/Corporate_spin-off
0
PAGE HAS NO LINKS: /wiki/Somers_Isles_Company
0
PAGE HAS NO LINKS: /wiki/London_and_Bristol_Company
0
PAGE HAS NO LINKS: /wiki/Cuper%27s_Cove
0
PAGE HAS NO LINKS: /wiki/St._John%27s,_Newfoundland_and_Labrador
0
PAGE HAS NO LINKS: /wiki/Charter_colony
0
PAGE HAS NO LINKS: /wiki/Bermuda
0
PAGE HAS NO LINKS: /wiki/British_Overseas_Territory
0
PAGE HAS NO LINKS: /wiki/Sea_Venture
0
PAGE HAS NO LINKS: /wiki/St._George%27s,_Bermuda
0
PAGE HAS NO LINKS: /wiki/New_World
0
PAGE HAS NO LINKS: /wiki/Privateer#Bermudians
0
PAGE HAS NO LINKS: /wiki/West_Indies
0
PAGE HAS NO LINKS: /wiki/Chesapeake_Bay
0
PAGE HAS NO LINKS: /wiki/New_England
0
PAGE HAS NO LINKS: /wiki/Barbados
487
0
PAGE HAS NO LINKS: /wiki/Royal_African_Company
0
PAGE HAS NO LINKS: /wiki/Atlantic_slave_trade
460
0
PAGE HAS NO LINKS: /wiki/Portugal
0
PAGE HAS NO LINKS: /wiki/Catherine_of_Braganza
0
PAGE HA

0
PAGE HAS NO LINKS: /wiki/Bantam_(city)
0
PAGE HAS NO LINKS: /wiki/East_Indies
0
PAGE HAS NO LINKS: /wiki/City_of_London
0
PAGE HAS NO LINKS: /wiki/East_India_Company
0
PAGE HAS NO LINKS: /wiki/Factory_(trading_post)
0
PAGE HAS NO LINKS: /wiki/Java
0
PAGE HAS NO LINKS: /wiki/Surat
0
PAGE HAS NO LINKS: /wiki/Bombay
0
PAGE HAS NO LINKS: /wiki/Machilipatnam
0
PAGE HAS NO LINKS: /wiki/Coromandel_Coast
0
PAGE HAS NO LINKS: /wiki/Run_(island)
0
PAGE HAS NO LINKS: /wiki/Nathaniel_Courthope
0
PAGE HAS NO LINKS: /wiki/Dutch_East_India_Company
0
PAGE HAS NO LINKS: /wiki/James_I_of_England
0
PAGE HAS NO LINKS: /wiki/Treaty_of_Westminster_(1654)
0
PAGE HAS NO LINKS: /wiki/Second_Anglo-Dutch_War
0
PAGE HAS NO LINKS: /wiki/Manhattan
0
PAGE HAS NO LINKS: /wiki/Fort_St._George_(India)
0
PAGE HAS NO LINKS: /wiki/Chennai
0
PAGE HAS NO LINKS: /wiki/George_Town,_Chennai
0
PAGE HAS NO LINKS: /wiki/Mumbai
487
0
PAGE HAS NO LINKS: /wiki/Catherine_of_Braganza
0
PAGE HAS NO LINKS: /wiki/John_IV_of_Portugal
0


0
PAGE HAS NO LINKS: /wiki/English_country_dance
0
PAGE HAS NO LINKS: /wiki/Architecture_of_England
0
PAGE HAS NO LINKS: /wiki/Anglo-Saxon_architecture
0
PAGE HAS NO LINKS: /wiki/English_Gothic_architecture
0
PAGE HAS NO LINKS: /wiki/Tudor_architecture
0
PAGE HAS NO LINKS: /wiki/Elizabethan_architecture
0
PAGE HAS NO LINKS: /wiki/Jacobean_architecture
0
PAGE HAS NO LINKS: /wiki/Queen_Anne_style_architecture
0
PAGE HAS NO LINKS: /wiki/Georgian_architecture
0
PAGE HAS NO LINKS: /wiki/National_symbols_of_England
0
PAGE HAS NO LINKS: /wiki/Flag_of_England
0
PAGE HAS NO LINKS: /wiki/List_of_English_flags
0
PAGE HAS NO LINKS: /wiki/English_heraldry
0
PAGE HAS NO LINKS: /wiki/Royal_Arms_of_England
0
PAGE HAS NO LINKS: /wiki/College_of_Arms
0
PAGE HAS NO LINKS: /wiki/Royal_Badges_of_England
0
PAGE HAS NO LINKS: /wiki/Royal_Supporters_of_England
0
PAGE HAS NO LINKS: /wiki/Royal_Standards_of_England
0
PAGE HAS NO LINKS: /wiki/Crown_Jewels_of_the_United_Kingdom
0
PAGE HAS NO LINKS: /wiki/Tudor_ro

0
PAGE HAS NO LINKS: /wiki/History_of_Somalia
0
PAGE HAS NO LINKS: /wiki/Sultanate_of_the_Geledi
0
PAGE HAS NO LINKS: /wiki/Majeerteen_Sultanate
0
PAGE HAS NO LINKS: /wiki/Sultanate_of_Hobyo
0
PAGE HAS NO LINKS: /wiki/Dervish_state
0
PAGE HAS NO LINKS: /wiki/Swedish_Empire
0
PAGE HAS NO LINKS: /wiki/Tu%CA%BBi_Tonga_Empire
0
PAGE HAS NO LINKS: /wiki/History_of_Turkey
0
PAGE HAS NO LINKS: /wiki/Ottoman_Empire
0
PAGE HAS NO LINKS: /wiki/Karamanids
0
PAGE HAS NO LINKS: /wiki/Ramadanids
0
PAGE HAS NO LINKS: /wiki/History_of_Vietnam
0
PAGE HAS NO LINKS: /wiki/Tay_Son_Dynasty
0
PAGE HAS NO LINKS: /wiki/Nguyen_Dynasty
0
PAGE HAS NO LINKS: /wiki/Empire_of_Vietnam
0
PAGE HAS NO LINKS: /wiki/Colonial_empire
0
PAGE HAS NO LINKS: /wiki/United_States_territorial_acquisitions
0
PAGE HAS NO LINKS: /wiki/Belgian_colonial_empire
0
PAGE HAS NO LINKS: /wiki/British_Empire
0
PAGE HAS NO LINKS: /wiki/Danish_colonial_empire
0
PAGE HAS NO LINKS: /wiki/Dutch_Empire
0
PAGE HAS NO LINKS: /wiki/French_colonial_em

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [ ]:
import smtplib
from email.mime.text import MIMEText

msg = MIMEText('The body of the email is here')

msg['Subject'] = 'An Email Alert'
msg['From'] = 'ryan@pythonscraping.com'
msg['To'] = 'webmaster@pythonscraping.com'

s = smtplib.SMTP('localhost')
s.send_message(msg)
s.quit()

In [1]:
import smtplib
from email.mime.text import MIMEText
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

def sendMail(subject, body):
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] ='christmas_alerts@pythonscraping.com'
    msg['To'] = 'ryan@pythonscraping.com'

    s = smtplib.SMTP('localhost')
    s.send_message(msg)
    s.quit()

bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
while(bs.find('a', {'id':'answer'}).attrs['title'] == 'NO'):
    print('It is not Christmas yet.')
    time.sleep(3600)
    bs = BeautifulSoup(urlopen('https://isitchristmas.com/'), 'html.parser')
sendMail('It\'s Christmas!', 
         'According to http://itischristmas.com, it is Christmas!')

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
'